In [ ]:
# Importing necessary libraries
from transformers import pipeline

# Loading the FinGPT model
model = pipeline('text-classification', model='AI4Finance/FinGPT-Large-Financial-News')

# Sample news article
news_article = "The stock market is expected to rise by 10% in the next quarter."

# Classifying the news article as real or fake
classification = model(news_article, return_all_scores=True)

# Printing the classification label and score
print(f"Classification: {classification[0]['label']}, Score: {classification[0]['score']}")


In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0

# Load Models
base_model = "NousResearch/Llama-2-13b-hf" 
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_8bit = False)
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()

# Make prompts
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .
Answer: ''',
]

# Generate results
tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512)
res = model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)

In [ ]:
def perform_sentiment_analysis():
    try:
        #Connect to DB
        collection = connect_to_mongodb()
        # Retrieve the articles with their titles
        articles = collection.find({}, {"_id": 1, "Content": 1})

        for article in articles:
            text = article["Centent"]

            # Generate results dynamically for each title
            prompt = [
                f'''Instruction: What is the sentiment of this news? Please choose an answer from {{negative/neutral/positive}}
                Input: {text}
                Answer: '''
            ]

            tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512, truncation=True)
            res = model.generate(**tokens, max_length=512)
            res_sentences = [tokenizer.decode(i) for i in res]
            out_text = [o.split("Answer: ")[1] for o in res_sentences]

            # Assuming binary sentiment classification (0 for negative, 1 for positive)
            sentiment_label = "positive" if out_text[0].lower() == 'positive' else "negative" if out_text[0].lower() == 'negative' else "neutral"

            # Update the collection with the sentiment label
            collection.update_one(
                {"_id": article["_id"]},
                {"$set": {"Sentiment": sentiment_label}}
            )

        print("Sentiment analysis and MongoDB update complete.")

    except Exception as e:
        print(f"An error occurred during sentiment analysis and update: {e}")

In [ ]:
L’un des principaux leviers pour atteindre nos objectifs climatiques, la rénovation énergétique des bâtiments, et en particulier des logements, est une priorité du Gouvernement. En 2024, les moyens consacrés aux aides à la rénovation énergétique des logements augmenteront significativement pour atteindre un niveau sans précédent de 5 milliards d’engagement, soit 1,6 Md€ de plus qu’en 2023.

Le renforcement de la lutte contre la fraude à la rénovation énergétique est le pendant de cette montée en puissance.

Plusieurs outils et acteurs sont mobilisés pour détecter, arrêter et sanctionner les fraudes à la rénovation. Des garde-fous ont été posés, pour par exemple interdire le démarchage téléphonique dans le secteur de la rénovation énergétique, et des programmes de contrôles, de natures complémentaires, sont menés chaque année. Au total environ 170 000 contrôles ont été opérés en 2022 et ont donné lieu à des sanctions administratives, voire pénales. En 2023, face à la détection de nouveaux schémas de fraude, les contrôles ont été maintenus à un niveau élevé et leur ciblage a été adapté à l’évolution des pratiques.

Avec le double objectif de protéger les Français et s’assurer du bon emploi de l’argent public, le Gouvernement a annoncé des évolutions structurantes dans l’attribution des aides pour limiter le risque de fraude :

- Depuis le 1er juin 2023, toute demande de financement MaPrimeRénov’ relative à un audit énergétique doit s’accompagner d’une demande de financement MaPrimeRénov’ relative aux travaux. Le schéma de fraudes à la réalisation de faux audits est ainsi maîtrisé.

- Dans le cadre de la refonte des aides à la rénovation énergétique, une obligation d’accompagnement par un « Accompagnateur Rénov’ », agrée par l’ANAH, est mise en place pour les aides les plus importantes.

- L’activité de mandataire financier sera davantage encadrée dès 2024. Le PLF pour 2024 (article 50) prévoit ainsi d’exiger des garanties financières plus fortes et une déclaration préalable auprès de l’Anah. Les dirigeants des entreprises mandataires condamnées pour des schémas frauduleux pourront être directement sanctionnés.

Les moyens de l’Etat dans la lutte contre la fraude à la rénovation énergétique seront encore renforcés autour de trois axes d’action :

D’une part, la protection des consommateurs sera encore accrue. Les effectifs de la DGCCRF seront doublés des dès 2024 (+24 emplois). Par ailleurs, la mise en place du filtre anti-arnaque, une des mesures phare du projet de loi pour sécuriser l’espace numérique, permettra de mieux protéger nos concitoyens en neutralisant les sites cybermalveillants et de hameçonnage qui usurpent l’identité et les signes visuels des services publics dédiés à la rénovation énergétique afin de soutirer des données personnelles sensibles ou générer des paiements sans contrepartie.

Les contrôles seront plus nombreux, mieux ciblés et plus transparents. 10% des dossiers MaPrimeRénov’ et 12,5% des dossiers au titre des aides CEE seront contrôlés sur place en 2024, contre 7 à 8% aujourd’hui. Le taux de contrôle par contact (courriel ou téléphone) augmentera de 20 % en 2024 puis 30 % en 2025. Ces contrôles se concentreront notamment sur les rénovations d’ampleur, en cohérence avec la priorité donnée à ces opérations dans le cadre du projet de loi de finances pour 2024. Par ailleurs, les services de l’Etat seront en mesure de cibler davantage leurs contrôles grâce au renforcement des outils informatiques de l’ANAH et la systématisation du partage d’information et des signalements dans le cadre de la mission interministérielle de lutte antifraude (MICAF). Enfin, la fiabilité des contrôles sera améliorée avec en particulier, le renforcement de l’indépendance des entreprises chargée du contrôle des travaux financés par les certificats d’économie d’énergie.

L’arsenal d’outils à la disposition des services de l’Etat sera renforcé. L’accès par l’ANAH au fichier des comptes bancaires permettra ainsi de mieux détecter la fraude et sera porté par un amendement au PLF 2024. D’autres mesures interviendront en 2024 pour renforcer le partage d’information ou encore, par exemple, permettre la suspension du versement des aides publiques en cas de suspicion de fraude organisée.

L’ensemble du Gouvernement est ainsi mobilisé et déterminé à accentuer la lutte contre la fraude à la rénovation énergétique.